In [2]:
import cv2 as cv
import numpy as np
import os

In [3]:
# function to display the coordinates of 
# of the points clicked on the image  
def click_event(event, x, y, flags, param): 
    
    corners, img, window = param
    # checking for left mouse clicks 
    if event == cv.EVENT_LBUTTONDOWN: 
            
        corners.append([x, y]) 
  
        # displaying the coordinates 
        # on the image window 
        font = cv.FONT_HERSHEY_SIMPLEX 
        cv.putText(img, 'X', (x,y), font, 
                    1, (255, 0, 0), 2) 
        cv.imshow(window, img)
  


In [ ]:
def run(auto= False, data= './data' ):

    images=  os.listdir(data) # get list of image filenames

    objp = np.zeros((7*6,3), np.float32)
    objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)
    objp= objp*16.5  # we measured the side of each square to be 16,5mm  

    # Lists to store object and image points from all images.
    objpoints = [] # 3d point in real world space
    imgpoints = [] # 2d points in image plane.

    criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

    for fname in images:

        img = cv.imread(f'{data}/{fname}')
        gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        corners = []
        window = f'image_{fname}'

        if auto:
        # Find the chess board corners
            ret, corners = cv.findChessboardCorners(gray, (7,6), None)
            if ret:        
                objpoints.append(objp)
                imgpoints.append(corners)
                
                # Draw and display the corners
                cv.drawChessboardCorners(img, (7,6), corners, True)
                cv.imshow(window, img)
                cv.waitKey(100)
            else:
                print(f'Pattern not found in {fname}')

            # If found, add object points, image points (after refining them)
        else:
            cv.namedWindow(window, cv.WINDOW_NORMAL)
            cv.imshow(window, img) 
            cv.setMouseCallback(window, click_event, [corners, img, window]) 
            # wait for a key to be pressed to exit 
            print("Click on the chessboard corners in order, then press any key.")
            cv.waitKey(0) 
            
            corners = np.array(corners, dtype=np.float32).reshape(-1, 2)  # Convert list to NumPy array
            objpoints.append(objp)
            imgpoints.append(corners)

        cv.destroyWindow(window)
            
    cv.destroyAllWindows()
    # print(f"Number of images: {len(imgpoints)}")
    # print(f"Shape of first image points: {imgpoints[0].shape if imgpoints else 'Empty'}")
    # print(f"Data type: {imgpoints[0].dtype if imgpoints else 'Empty'}")
    ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape, None, None)

    return mtx, dist, rvecs, tvecs

In [5]:
run()


Click on the chessboard corners in order, then press any key.
Number of images: 1
Shape of first image points: (42, 2)
Data type: float32


(array([[2.50047866e+03, 0.00000000e+00, 4.58789512e+02],
        [0.00000000e+00, 4.51055431e+03, 7.16299392e+02],
        [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
 array([[ 5.93819830e+00, -9.33960469e+01,  2.10627685e-01,
         -1.92124891e-01,  9.48857175e+02]]),
 (array([[ 1.50013309],
         [ 1.94947383],
         [-1.03946241]]),),
 (array([[  26.66310041],
         [-117.44283412],
         [ 857.99021706]]),))

In [9]:
mtx

array([[2.82877149e+03, 0.00000000e+00, 4.23686216e+02],
       [0.00000000e+00, 3.08417159e+03, 5.66067314e+02],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])